In [83]:
# This NoteBook will perform best model selection for Random Forest Algorithm. 

In [84]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn # scatterplot
from IPython.display import display

In [ ]:
# Read in the dataset
mob = pd.read_csv("Datasets/MobileData.csv")

In [69]:
# Glimpse into the object types
print("Object types: \n{}".format(mob.dtypes))

# This dataset is fully numeric. We have a mixture of float and integer dtypes

Object types: 
battery_power      int64
blue               int64
clock_speed      float64
dual_sim           int64
fc                 int64
four_g             int64
int_memory         int64
m_dep            float64
mobile_wt          int64
n_cores            int64
pc                 int64
px_height          int64
px_width           int64
ram                int64
sc_h               int64
sc_w               int64
talk_time          int64
three_g            int64
touch_screen       int64
wifi               int64
price_range        int64
dtype: object


In [70]:
# Glimpse 
mob.describe()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
count,2000.000000,2000.0000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1238.518500,0.4950,1.522250,0.509500,4.309500,0.521500,32.046500,0.501750,140.249000,4.520500,...,645.108000,1251.515500,2124.213000,12.306500,5.767000,11.011000,0.761500,0.503000,0.507000,1.500000
std,439.418206,0.5001,0.816004,0.500035,4.341444,0.499662,18.145715,0.288416,35.399655,2.287837,...,443.780811,432.199447,1084.732044,4.213245,4.356398,5.463955,0.426273,0.500116,0.500076,1.118314
min,501.000000,0.0000,0.500000,0.000000,0.000000,0.000000,2.000000,0.100000,80.000000,1.000000,...,0.000000,500.000000,256.000000,5.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,851.750000,0.0000,0.700000,0.000000,1.000000,0.000000,16.000000,0.200000,109.000000,3.000000,...,282.750000,874.750000,1207.500000,9.000000,2.000000,6.000000,1.000000,0.000000,0.000000,0.750000
50%,1226.000000,0.0000,1.500000,1.000000,3.000000,1.000000,32.000000,0.500000,141.000000,4.000000,...,564.000000,1247.000000,2146.500000,12.000000,5.000000,11.000000,1.000000,1.000000,1.000000,1.500000
75%,1615.250000,1.0000,2.200000,1.000000,7.000000,1.000000,48.000000,0.800000,170.000000,7.000000,...,947.250000,1633.000000,3064.500000,16.000000,9.000000,16.000000,1.000000,1.000000,1.000000,2.250000
max,1998.000000,1.0000,3.000000,1.000000,19.000000,1.000000,64.000000,1.000000,200.000000,8.000000,...,1960.000000,1998.000000,3998.000000,19.000000,18.000000,20.000000,1.000000,1.000000,1.000000,3.000000


In [93]:
def find_max(lst):
    vmax = 0
    for ele in lst:
        if ele > vmax:
            vmax = ele
    return vmax

In [122]:

def test(data):
    for i in range(len(data)):
        x = 0
        if(i == 100):
            x+=1
    return x

print(test(mob))

0


In [ ]:
# Let's separate categorical variables now.
# We need to see what is included in the dataset.

def find_categoricals(data):
    categoricals = []
    for variable in data.columns:
        if(max(j) == 1):
            categoricals.append(variable)
    return categoricals

99

In [ ]:
# Let's create a scatterplot
from pandas.plotting import scatter_matrix

mob_df = pd.DataFrame(data=mob, columns=mob.columns)

grr = scatter_matrix(mob_df, c=y, figsize=(15,15), 
                     marker='o', hist_kwds={'bins' : 20}, 
                     s = 60, alpha = 0.8, cmap = mglearn.cm3)